In [1]:
using DifferentialEquations
using Distances
using PyCall
@pyimport matplotlib.pyplot as plt
using GpAbc

# Rejection ABC using GaussProABC - estimating parameters

This notebook demonstrates how to perform simulation- and emulation-based rejection ABC using GaussProABC for parameter estimation of an ODE model.

Start by choosing some settings for ABC, the emulator and the toy, noise-free ODE system. There is also a wrapper function that returns the solution to the toy system for some given parameters and ODE solution options.

In [3]:
#
# ABC settings
#
n_var_params = 2
n_particles = 1000
threshold = 0.5
priors = [Distributions.Uniform(0., 5.), Distributions.Uniform(0., 5.)]
distance_metric = euclidean
progress_every = 1000

#
# Emulation settings
#
n_design_points = 100
batch_size = 1000
max_iter = 1000

#
# True parameters
#
true_params =  [2.0, 1.0, 15.0, 1.0, 1.0, 1.0, 100.0, 1.0, 1.0, 1.0]

#
# ODE solver settings
#
Tspan = (0.0, 10.0)
x0 = [3.0, 2.0, 1.0]
solver = RK4()
saveat = 0.1

#
# Returns the solution to the toy model as solved by DifferentialEquations
#
GeneReg = function(params::AbstractArray{Float64,1},
    Tspan::Tuple{Float64,Float64}, x0::AbstractArray{Float64,1},
    solver::OrdinaryDiffEq.OrdinaryDiffEqAlgorithm, saveat::Float64)

  if size(params,1) != 10
    throw(ArgumentError("GeneReg needs 10 parameters, $(size(params,1)) were provided"))
  end

  function ODE_3GeneReg(dx, x, par, t)
    dx[1] = par[1]/(1+par[7]*x[3]) - par[4]*x[1]
    dx[2] = par[2]*par[8]*x[1]./(1+par[8]*x[1]) - par[5]*x[2]
    dx[3] = par[3]*par[9]*x[1]*par[10]*x[2]./(1+par[9]*x[1])./(1+par[10]*x[2]) - par[6]*x[3]
  end

  prob = ODEProblem(ODE_3GeneReg, x0 ,Tspan, params)
  Obs = solve(prob, solver, saveat=saveat)

  return Obs
end

(::#1) (generic function with 1 method)

## Plot reference data

The reference data is the concentrations over time for the three species. Computed by solving the model with the true parameters.

In [4]:
#
# Get reference data and plot it
#
reference_data = GeneReg(true_params, Tspan, x0, solver, saveat)
println("Got reference data")

fig, ax = plt.subplots(1)
ax[:plot](reference_data.t, transpose(hcat(reference_data.u...)))
ax[:set_xlabel]("t")
ax[:set_ylabel]("C(t)")
plt.show()

Got reference data


## Simulation-based rejection ABC

To perform rejection ABC, first construct an instance of one of either two objects: `SimulatedABCRejectionInput` or `EmulatedABCRejectionInput`.

The simulated version contains the number of accepted particles, acceptance threshold, priors, the distance metric and the simulator funciton. Note that this needs to be extended to include summary statistics.

The simulator function is user-defined and must to take the unknown parameters as a single argument and return the simluated trajectories.

After constructing the input object it is passed to `ABCrejection` with the reference (observed) data to perform rejection ABC.

In [5]:
#
# Simulation
#
simulator_function(var_params) = GeneReg(vcat(var_params, true_params[n_var_params+1:end]), Tspan, x0, solver, saveat)

sim_rej_input = SimulatedABCRejectionInput(n_var_params,
                        n_particles,
                        threshold,
                        priors,
                        distance_metric,
                        simulator_function)

sim_result = ABCrejection(sim_rej_input, reference_data, progress_every=progress_every)

2018-06-18T12:34:01.698 Accepted 35/1000 particles.
2018-06-18T12:34:01.817 Accepted 60/2000 particles.
2018-06-18T12:34:01.92 Accepted 81/3000 particles.
2018-06-18T12:34:02.023 Accepted 97/4000 particles.
2018-06-18T12:34:02.125 Accepted 124/5000 particles.
2018-06-18T12:34:02.228 Accepted 157/6000 particles.
2018-06-18T12:34:02.328 Accepted 182/7000 particles.
2018-06-18T12:34:02.432 Accepted 201/8000 particles.
2018-06-18T12:34:02.534 Accepted 218/9000 particles.
2018-06-18T12:34:02.636 Accepted 236/10000 particles.
2018-06-18T12:34:02.739 Accepted 258/11000 particles.
2018-06-18T12:34:02.839 Accepted 275/12000 particles.
2018-06-18T12:34:02.942 Accepted 295/13000 particles.
2018-06-18T12:34:03.053 Accepted 318/14000 particles.
2018-06-18T12:34:03.181 Accepted 335/15000 particles.
2018-06-18T12:34:03.295 Accepted 368/16000 particles.
2018-06-18T12:34:03.405 Accepted 389/17000 particles.
2018-06-18T12:34:03.516 Accepted 404/18000 particles.
2018-06-18T12:34:03.632 Accepted 429/19000

GpAbc.ABCRejectionOutput(2, 1000, 41774, 0.5, [1.86983 1.03419; 1.07104 0.989697; … ; 2.12076 1.08467; 3.28688 0.998401], [0.116237, 0.417295, 0.458917, 0.403296, 0.407367, 0.408753, 0.226735, 0.332124, 0.329913, 0.386028  …  0.281795, 0.401663, 0.467238, 0.216794, 0.281798, 0.148056, 0.440837, 0.452263, 0.296646, 0.454594], [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001  …  0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001])

## Emulation-based Rejection ABC

First train the emulator. This requires training inputs `X` and training outputs `y`, where `X` is an array with size (n_design_points, n_unknown_parameters) and `y` is an array with size (n_design_points).

The function `get_training_data` returns `X` and `y` in the correct format to be used with a `GPModel`.

Then train the model, plotting the design points and their distances.

In [6]:
#
# Train the Emulator
#

#
# First prepare data for emulator - matrix X contains design ponts and y contains distances
#
function get_training_data(n_design_points,
  priors,
  simulator_function, distance_metric,
  reference_data)
    
  X = zeros(n_design_points, length(priors))
  y = zeros(n_design_points)  
  for i in 1:n_design_points
    dp = [rand(d) for d in priors]
    X[i,:] = dp
    y[i] = distance_metric(simulator_function(dp), reference_data)
  end
    
  return X, y
end

X, y = get_training_data(n_design_points, priors, simulator_function, distance_metric, reference_data)

#
# Train emulator - VARIANCE SEEMS SMALL
#
gpem = GPModel(training_x=X, training_y=y, kernel=SquaredExponentialArdKernel())
gp_train(gpem)

println("Trained emulator")

#
# Plot design points
#
fig, axes = plt.subplots(n_var_params)

for (idx, prior) in enumerate(priors)
    ax = axes[idx]
    ax[:scatter](X[:,idx], y, label="training data")
    ax[:set_xlabel]("Parameter $idx")
    ax[:set_ylabel]("Distance")
end

plt.tight_layout()
plt.show()

Trained emulator


The second parameter is easier to pick up using this distance metric and summary statistic than the first.

When creating an `EmulatedABCRejectionInput` object provide a function that returns the emulated distances rather than the `simulator_function` required in the simulation case. Also provide a batch size and a maximum number of iterations.

In [8]:
#
# Use the emulator for rejection ABC
#
function predict_distance(p::AbstractArray{Float64})
    result = gp_regression(p,gpem)[1]
    return result
end

emu_rej_input = EmulatedABCRejectionInput(n_var_params,  
  n_particles,
  threshold,
  priors,
  predict_distance,
  batch_size,
  max_iter)

emu_result = ABCrejection(emu_rej_input, reference_data, progress_every=5)

2018-06-18T12:36:45.431 Accepted 147/5000 particles (5 batches of size 1000).
2018-06-18T12:36:45.479 Accepted 290/10000 particles (10 batches of size 1000).
2018-06-18T12:36:45.522 Accepted 429/15000 particles (15 batches of size 1000).
2018-06-18T12:36:45.578 Accepted 589/20000 particles (20 batches of size 1000).
2018-06-18T12:36:45.627 Accepted 724/25000 particles (25 batches of size 1000).
2018-06-18T12:36:45.695 Accepted 883/30000 particles (30 batches of size 1000).


GpAbc.ABCRejectionOutput(2, 1000, 34000, 0.5, [3.12865 0.974133; 1.72081 1.03998; … ; 2.77749 1.0673; 1.53748 1.08761], [0.380466, 0.265027, 0.440093, 0.404073, 0.39671, 0.445372, 0.284085, 0.371401, 0.236868, 0.364189  …  0.486624, 0.268176, 0.328258, 0.284831, 0.256291, 0.462372, 0.488637, 0.432112, 0.367073, 0.350106], [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001  …  0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001])

# Comparing the resulting posteriors from simulation and emulation

They are similar and both center on the correct parameter values.

In [9]:
fig, axes = plt.subplots(n_var_params)

for idx in 1:n_var_params
    ax = axes[idx]
    ax[:hist](emu_result.population[:,idx], normed=true, label="emulation", alpha=0.5)
    ax[:hist](sim_result.population[:,idx], normed=true, label="simlation", alpha=0.5)
    ax[:set_xlabel]("Parameter $idx")
    ax[:set_ylabel]("Density")
end

axes[1][:legend]()
plt.tight_layout()
plt.show()

# ABC-SMC

In [10]:
#
# ABC-SMC settings
#
threshold_schedule = [3.0, 2.0, 1.0]

3-element Array{Float64,1}:
 3.0
 2.0
 1.0

## Simulation-based ABC-SMC

In [11]:
sim_abcsmc_input = SimulatedABCSMCInput(n_var_params,
    n_particles,
    threshold_schedule,
    priors,
    distance_metric,
    simulator_function)

sim_abcsmc_res = ABCSMC(sim_abcsmc_input, reference_data)

2018-06-18T12:40:20.302 Accepted 357/1000 particles.
2018-06-18T12:40:20.412 Accepted 714/2000 particles.
2018-06-18T12:40:21.633 Accepted 423/1000 particles.
2018-06-18T12:40:22.127 Accepted 814/2000 particles.
2018-06-18T12:40:22.966 Accepted 245/1000 particles.
2018-06-18T12:40:23.464 Accepted 528/2000 particles.
2018-06-18T12:40:23.954 Accepted 764/3000 particles.


GpAbc.ABCSMCOutput(2, [1000, 1000, 1000], [2857, 2518, 3881], [3.0, 2.0, 1.0], Array{Float64,2}[[0.944045 0.543657; 3.77091 1.74749; … ; 2.6858 1.67859; 3.53236 0.315912], [3.23743 0.748698; 0.493678 1.44457; … ; 1.01928 1.53161; 3.66575 0.801198], [0.233936 1.08984; 3.59796 0.760199; … ; 2.15645 1.23355; 1.75841 0.843716]], Array{Float64,1}[[1.76071, 2.58433, 0.802846, 1.31407, 2.20297, 2.74513, 1.90958, 1.34824, 0.842735, 0.732341  …  1.03709, 0.835176, 1.55099, 1.91284, 1.88133, 1.03907, 0.0902711, 2.09996, 2.20611, 2.57852], [0.895412, 1.49463, 1.60115, 0.906943, 1.71723, 0.790514, 0.508649, 1.97667, 1.44579, 1.03369  …  1.3578, 0.694482, 1.5939, 0.870163, 0.972272, 1.74365, 0.595101, 1.90084, 1.65374, 0.788352], [0.963439, 0.89395, 0.539074, 0.810967, 0.858149, 0.530326, 0.113588, 0.6417, 0.696302, 0.737659  …  0.865686, 0.930452, 0.438482, 0.713755, 0.980899, 0.148138, 0.790147, 0.0803972, 0.782455, 0.564746]], StatsBase.Weights[[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0

## Emulation-based ABC-SMC

In [12]:
emu_abcsmc_input = EmulatedABCSMCInput(n_var_params,
    n_particles,
    threshold_schedule,
    priors,
    predict_distance,
    batch_size,
    max_iter)

emu_abcsmc_res = ABCSMC(emu_abcsmc_input, reference_data)

2018-06-18T12:40:39.737 Accepted 636/1000 particles.
2018-06-18T12:40:40.436 Accepted 356/1000 particles.
2018-06-18T12:40:40.874 Accepted 724/2000 particles.


GpAbc.ABCSMCOutput(2, [1000, 1000, 1000], [3000, 1589, 2752], [3.0, 2.0, 1.0], Array{Float64,2}[[3.10599 1.19705; 0.165989 0.707074; … ; 3.41025 0.654842; 1.50906 1.78178], [3.67448 1.24432; 4.91527 1.28832; … ; 2.69262 1.0116; 2.79549 1.29598], [0.929949 1.2148; 2.27616 1.19188; … ; 2.42213 1.30101; 2.08102 1.14962]], Array{Float64,1}[[0.749961, 1.73866, 0.480651, 2.68946, 2.90026, 2.37717, 1.30082, 1.81033, 1.53606, 1.23875  …  2.23786, 0.755177, 1.95483, 1.89138, 0.675651, 1.91647, 0.552367, 2.88897, 1.21527, 2.39759], [1.05392, 1.53437, 0.430137, 0.599135, 0.441277, 1.37643, 1.83888, 0.55937, 0.466769, 1.27648  …  1.53424, 0.320213, 0.574913, 1.61465, 0.527314, 0.706582, 1.74474, 0.54402, 0.290858, 1.00934], [0.790834, 0.583187, 0.518773, 0.543667, 0.530658, 0.593196, 0.767602, 0.948973, 0.991596, 0.57066  …  0.358682, 0.675023, 0.632226, 0.999097, 0.728539, 0.690469, 0.437596, 0.404827, 0.969403, 0.450669]], StatsBase.Weights[[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001

## Compare the posteriors

In [13]:
abcsmc_res = [sim_abcsmc_res, emu_abcsmc_res]

fig, axes = plt.subplots(2,n_var_params)

for i in 1:2
    for j in 1:n_var_params
        for k in 1:length(threshold_schedule)
            axes[i,j][:hist](abcsmc_res[i].population[k][:,j],
            label = "Run $k",
            normed=true, alpha=0.3)
        end
    end
end

axes[1,1][:legend]()
axes[1,1][:set_ylabel]("Simulation")
axes[2,1][:set_ylabel]("Emulation")
for j in 1:n_var_params
    axes[2,j][:set_xlabel]("Parameter $j")
end



plt.tight_layout()
plt.show()

In [21]:
size(abcsmc_res[1].population[end])

(1000, 2)